Progress Report 1
----

**Team Members**

Yaqian Cheng, Department of Statistical Science

Mengrun Li, Department of Statistical Science

**Github repository**

<https://github.com/cici7941/Sta_663_Statistical_Computation_Final_Project>

**Choice of paper** 

*Scalable K-Means++*

**Abstract**

*K-means* is one of the most popular clustering methods. A good initialization of *k-means* is essential for obtaining the global optimal solution and efficiency. However, there are two main obstacles with traditional *k-means* method. One is theoretical inefficiency and the other one is that its final solution is locally optimal. A better algorithm, *k-means++* addresses the second problem with an improved initialization procedure of the cluster centers. But this *k-means++* initialization is not parallelizable, because the selection for the *i*th center depends on the previous *i-1* centers [1]. Therefore, *k-means||*, a parallelizable version of *k-means++*, has been raised, which can both improve the final solution and run faster. In this report, we implemented the algorithm in the paper "Scalable K-Means++" in Python, compared the clustering cost and runtime between *k-means*, *k-means++* and *k-means||*, performed tests for main functions, profiled the performance of the algorithm and identified bottlenecks, and performed optimization using Cython. We then apply *k-means||* to a massive dataset to evaluate its performance.

**Outline**

1. Introduction
2. Algorithm  
    2.1 K-Means  
    2.2 K-Means++  
    2.3 K-Means||  
3. Code Testing
4. Profiling and Optimization
5. Application and Comparison

In [99]:
import scipy.linalg as la
import numpy as np

In [103]:
# helper functions
def euc_dist(x, y):
    return la.norm(x-y)

def centroid(X):
    return X.mean(0)
    
def d(x, Y):
    minDist = float("inf")
    for yi in Y:
        dist = euc_dist(x, yi)
        if(dist < minDist):
            minDist = dist
    return minDist

def cost(Y, C):
    cost = 0
    for yi in Y:
        cost += d(yi, C)**2
    return cost

In [167]:
# K-Means++
def kmeansPlus(data, k):
    idx = np.random.choice(data.shape[0], 1)
    C = data[idx, :]
    while(C.shape[0] < k):
        prob = [d(xi,C)**2 for xi in data]/cost(data,C)
        new = data[np.random.choice(data.shape[0], size=1, p=prob),:]
        if(new.tolist() not in C.tolist()):
            C = np.r_[C, new]
    return C

In [168]:
# K-Means||
X_ori = np.array([[1,0],[2,0],[3,5],[7,4],[6,8],[4,6],[9,12],[2,2],[4,1]])
X = X_ori
k = 3
l = 2
##step1
##Sample a point uniformly at random from X
idx1 = np.random.choice(X_ori.shape[0],1,replace = False)
C = X[idx1-1,:]
print(idx1)
print("first centroid:",C)
##step2
##initial cost
gamma1 = cost(X,C)
print("initial cost:",gamma1)
##remove first centroid
X = np.delete(X, idx1-1, axis=0)
X

[6]
first centroid: [[4 6]]
initial cost: 214.0


array([[ 1,  0],
       [ 2,  0],
       [ 3,  5],
       [ 7,  4],
       [ 6,  8],
       [ 9, 12],
       [ 2,  2],
       [ 4,  1]])

In [169]:
##step3
for i in range(int(round(np.log(gamma1)))):
    gamma = cost(X,C)
    prob = [l*d(x,C)**2 for x in X]/gamma
    new = X[np.random.choice(X.shape[0], size=1, p=prob),:]
    if(new.tolist() not in C.tolist()):
            C = np.r_[C, new]
    #X = np.delete(X, idx, axis = 0)
    #C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

ValueError: probabilities do not sum to 1

In [170]:
##step3
for i in range(int(round(np.log(gamma1)))):
    Ct = []
    idx = []
    gamma = cost(X,C)
    prob = [d(x,C)**2 for x in X]/gamma
    #new = data[np.random.choice(X.shape[0], size=1, p=prob),:]
    for j in range(len(X)):
        point = np.random.binomial(1,prob[j],size = 1)
        if point == 1:
           Ct.append(X[j,:])
           idx.append(j)
    X = np.delete(X, idx, axis = 0)
    C = np.concatenate((C,np.array(Ct).reshape(len(Ct),2)),axis = 0)
C

TypeError: unsupported operand type(s) for /: 'list' and 'float'